In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.externals import joblib
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
import gc
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier

%matplotlib inline

In [2]:
plt.rc('figure',dpi=100);

In [3]:
avisos = pd.read_csv("../../Data/fiuba_entrenamiento/gian/avisosONEHOT.csv")
postulaciones = pd.read_csv("../../Data/fiuba_entrenamiento/gian/postulaciones.csv")
postulantes = pd.read_csv("../../Data/fiuba_entrenamiento/gian/postulantesONEHOT.csv")
areas=avisos['nombre_area'].value_counts()
avisos['nombre_area']=avisos['nombre_area'].replace(areas.index, areas.values)

avisos_no_one_hot= pd.read_csv("../../Data/fiuba_entrenamiento/gian/avisos.csv")
avisos_no_one_hot=avisos_no_one_hot[['idaviso','tipo_de_trabajo','nivel_laboral','nombre_area']];
avisos_no_one_hot['tipo_de_trabajo']=avisos_no_one_hot['tipo_de_trabajo'].astype('category')
avisos_no_one_hot['nivel_laboral']=avisos_no_one_hot['nivel_laboral'].astype('category')
avisos_no_one_hot['nombre_area']=avisos_no_one_hot['nombre_area'].astype('category')
avisos_no_one_hot=avisos_no_one_hot.rename(columns={'nombre_area': 'nombre_area_aux'})

postulaciones['sepostulo']=1
postulaciones.drop(['fechapostulacion','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)

In [6]:
vistas = pd.read_csv("../../Data/fiuba_entrenamiento/gian/vistas.csv")
vistas.drop(['timestamp','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)
vistas=vistas.rename(columns={'idAviso': 'idaviso'})
vistas.head()

,idaviso,idpostulante
0,1111780242,YjVJQ6Z
1,1112263876,BmVpYoR
2,1112327963,wVkBzZd
3,1112318643,OqmP9pv
4,1111903673,DrpbXDP


In [7]:
vistas_por_area=vistas.copy()
vistas_por_area['vistas_por_area']=0;
vistas_por_area=pd.merge(vistas_por_area,avisos_no_one_hot,on='idaviso', how='left')
vistas_por_area=vistas_por_area.groupby(['idpostulante','nombre_area_aux'],as_index=False).agg({'vistas_por_area':'count'})
vistas_por_area.head()

,idpostulante,nombre_area_aux,vistas_por_area
0,0005E,Tecnologia / Sistemas,2
1,00Lkv,Comercio Exterior,2
2,00dMd,Comercio Exterior,4
3,01QAq,Contabilidad,1
4,021OM,Comercio Exterior,5


In [8]:
vistas_por_nivel=vistas.copy()
vistas_por_nivel['vistas_por_nivel']=0;
vistas_por_nivel=pd.merge(vistas_por_nivel,avisos_no_one_hot,on='idaviso', how='left')
vistas_por_nivel=vistas_por_nivel.groupby(['idpostulante','nivel_laboral'],as_index=False).agg({'vistas_por_nivel':'count'})
vistas_por_nivel.head()

,idpostulante,nivel_laboral,vistas_por_nivel
0,0005E,Otro,2
1,00Lkv,Otro,2
2,00dMd,Otro,4
3,01QAq,Otro,1
4,021OM,Otro,5


In [12]:
vistas_por_tipo=vistas.copy()
vistas_por_tipo['vistas_por_tipo']=0;
vistas_por_tipo=pd.merge(vistas_por_tipo,avisos_no_one_hot,on='idaviso', how='left')
vistas_por_tipo=vistas_por_tipo.groupby(['idpostulante','tipo_de_trabajo'],as_index=False).agg({'vistas_por_tipo':'count'})
vistas_por_tipo.head()

,idpostulante,tipo_de_trabajo,vistas_por_tipo
0,0005E,Full-time,2
1,00Lkv,Full-time,2
2,00dMd,Full-time,4
3,01QAq,Full-time,1
4,021OM,Full-time,5


In [14]:
vistas['cantidad']=0
vistas2=vistas.groupby(['idaviso','idpostulante'],as_index=False).agg({'cantidad':'count'})
vistas2.head()

,idaviso,idpostulante,cantidad
0,18,BolNL,2
1,48375,RwVdKR,1
2,169730,1KjXB,2
3,169730,2AKzxa,2
4,169730,6LJ64,1


In [15]:
vistas=vistas.groupby(['idpostulante'],as_index=False).agg({'cantidad':'count'});
vistas.head()

,idpostulante,cantidad
0,0002q,15
1,0005E,58
2,000R8,14
3,001XE,26
4,003k9,32


In [16]:
vistas3=pd.merge(vistas2,vistas,on='idpostulante',how='left')

In [17]:
vistas3.head()

,idaviso,idpostulante,cantidad_x,cantidad_y
0,18,BolNL,2,28
1,48375,RwVdKR,1,15
2,169730,1KjXB,2,99
3,169730,2AKzxa,2,2
4,169730,6LJ64,1,23


In [18]:
vistas3['cantidad']=vistas3['cantidad_x']/vistas3['cantidad_y']
vistas3.drop(['cantidad_x','cantidad_y'],axis=1,inplace=True)
vistas3.head()

,idaviso,idpostulante,cantidad
0,18,BolNL,0.071429
1,48375,RwVdKR,0.066667
2,169730,1KjXB,0.020202
3,169730,2AKzxa,1.000000
4,169730,6LJ64,0.043478


In [19]:
vistas=vistas.rename(columns={'cantidad': 'vistas_postulante'})
vistas2=vistas2.rename(columns={'cantidad': 'vistas_al_aviso'})

In [20]:
postulaciones2= pd.read_csv("../../Data/fiuba_entrenamiento/gian/no-postulaciones/7.csv")
postulaciones2=postulaciones2.append(pd.read_csv("../../Data/fiuba_entrenamiento/gian/no-postulaciones/8.csv").sample(len(postulaciones2)//2).reset_index(drop=True))
print(len(postulaciones2))
postulaciones2

9906801


,idaviso,idpostulante,sepostulo
0,1112389495,5NbOjG,0
1,1112024109,1QP841b,0
2,1112490884,96XQ6N9,0
3,1111474259,qe2QJj1,0
4,1112211292,KBzbBJm,0
5,1112328485,3NzZv26,0
6,1112474766,akDjWwE,0
7,1112312385,aZpBJW,0
8,1112285628,akDWj55,0
9,1112322843,GNd0KNN,0


In [21]:
postulaciones.head()

,idaviso,idpostulante,sepostulo
0,1112257047,NM5M,1
1,1111920714,NM5M,1
2,1112346945,NM5M,1
3,1112345547,NM5M,1
4,1112237522,5awk,1


In [22]:
postulaciones=postulaciones.append(postulaciones2, ignore_index=True)
print(len(postulaciones))
postulaciones2=0
gc.collect()
postulaciones.drop_duplicates(['idaviso','idpostulante'],keep='first',inplace=True)
print(len(postulaciones))

16511335
16501958


In [23]:
postulaciones=pd.merge(postulaciones,postulantes,on='idpostulante')
postulaciones=pd.merge(postulaciones,avisos,on='idaviso')
postulaciones=pd.merge(postulaciones,avisos_no_one_hot,on='idaviso')
postulaciones=pd.merge(postulaciones,vistas,on=['idpostulante'],how='left')
postulaciones=pd.merge(postulaciones,vistas2,on=['idaviso','idpostulante'],how='left')
postulaciones=pd.merge(postulaciones,vistas3,on=['idaviso','idpostulante'],how='left')
postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,edad,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,Doctorado_Abandonado,Doctorado_En Curso,Doctorado_Graduado,...,nivel_laboral_Jefe / Supervisor / Responsable,nivel_laboral_Junior,nivel_laboral_Otro,nivel_laboral_Senior / Semi-Senior,tipo_de_trabajo,nivel_laboral,nombre_area_aux,vistas_postulante,vistas_al_aviso,cantidad
3971366,1112371366,Rz6bv16,0,41.0,True,False,False,False,False,False,...,False,True,False,False,Full-time,Junior,Contabilidad,15.0,NaN,NaN
10610450,1112414499,96WjLev,0,30.0,False,True,False,False,False,False,...,False,False,False,True,Full-time,Senior / Semi-Senior,Redes,NaN,NaN,NaN
13594097,1112288883,6rBPqXl,1,24.0,True,False,False,False,False,False,...,False,False,True,False,Full-time,Otro,Ventas,31.0,2.0,0.064516
5256118,1112371554,ZqVDa1,1,44.0,False,True,False,False,False,False,...,False,False,True,False,Full-time,Otro,Atención al Cliente,30.0,NaN,NaN
14488072,1112116738,YaNVvJ,0,26.0,True,False,False,False,False,False,...,False,True,False,False,Full-time,Junior,Jóvenes Profesionales,3.0,NaN,NaN
8603889,1112251605,GNZ0rRe,0,46.0,False,True,False,False,False,False,...,False,False,False,True,Full-time,Senior / Semi-Senior,Ventas,NaN,NaN,NaN
13704818,1112397143,EBVvJ6,1,40.0,False,True,False,False,False,False,...,False,False,False,True,Full-time,Senior / Semi-Senior,Seguridad,35.0,7.0,0.200000
9506741,1112448235,BmaxmWE,0,24.0,False,True,False,False,False,False,...,True,False,False,False,Full-time,Jefe / Supervisor / Responsable,Ingeniería Industrial,NaN,NaN,NaN
9271052,1112501541,X953VJd,0,200.0,False,False,True,False,False,False,...,True,False,False,False,Teletrabajo,Jefe / Supervisor / Responsable,Tecnologia / Sistemas,1.0,NaN,NaN
5170856,1112291118,lD28BeX,1,23.0,True,False,False,False,False,False,...,False,False,False,True,Part-time,Senior / Semi-Senior,Venta de Seguros,NaN,NaN,NaN


In [24]:
postulaciones['cantidad']=postulaciones['cantidad'].fillna(0)
postulaciones['vistas_postulante']=postulaciones['vistas_postulante'].fillna(0)
postulaciones['vistas_al_aviso']=postulaciones['vistas_al_aviso'].fillna(0)
postulaciones['lo_vio']=(postulaciones['vistas_al_aviso']>0)
postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,edad,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,Doctorado_Abandonado,Doctorado_En Curso,Doctorado_Graduado,...,nivel_laboral_Junior,nivel_laboral_Otro,nivel_laboral_Senior / Semi-Senior,tipo_de_trabajo,nivel_laboral,nombre_area_aux,vistas_postulante,vistas_al_aviso,cantidad,lo_vio
9674621,1112263307,JBx5qAk,1,27.0,True,False,False,False,False,False,...,False,True,False,Full-time,Otro,Ventas,4.0,0.0,0.000000,False
11488760,1112498854,NMWa85,0,42.0,True,False,False,False,False,False,...,False,False,True,Full-time,Senior / Semi-Senior,Producción,59.0,0.0,0.000000,False
14055368,1112374844,pzeb1jm,1,24.0,False,True,False,False,False,False,...,True,False,False,Full-time,Junior,Ventas,113.0,0.0,0.000000,False
5580819,1112413019,vVjxWlX,1,19.0,False,True,False,False,False,False,...,True,False,False,Full-time,Junior,Recursos Humanos,98.0,0.0,0.000000,False
9404264,1112372438,bOjKG0B,0,19.0,False,True,False,False,False,False,...,False,False,True,Full-time,Senior / Semi-Senior,Ventas,4.0,0.0,0.000000,False
6150888,1263850,wVYlXdm,0,29.0,True,False,False,False,False,False,...,False,True,False,Por Contrato,Otro,Administración,8.0,0.0,0.000000,False
5747983,1112313865,xkavZPV,1,27.0,False,True,False,False,False,False,...,False,False,True,Full-time,Senior / Semi-Senior,Almacén / Depósito / Expedición,212.0,0.0,0.000000,False
7797190,1112455260,qe2OAJ1,0,21.0,True,False,False,False,False,False,...,False,False,True,Full-time,Senior / Semi-Senior,Programación,0.0,0.0,0.000000,False
10828462,1112266635,Z2DwKZ,0,38.0,True,False,False,False,False,False,...,False,False,True,Full-time,Senior / Semi-Senior,Finanzas,91.0,0.0,0.000000,False
3461768,1112259001,eaYBaW,1,32.0,False,True,False,False,False,False,...,False,False,True,Full-time,Senior / Semi-Senior,Marketing,114.0,0.0,0.000000,False


In [25]:
postulaciones_por_area=postulaciones.copy()
postulaciones_por_area=postulaciones_por_area.groupby(by=['idpostulante','nombre_area_aux'],as_index=False).agg({'sepostulo':'count'})
postulaciones_por_area=postulaciones_por_area.rename(columns={'sepostulo':'postulaciones_misma_area'})
postulaciones_por_area.head()

,idpostulante,nombre_area_aux,postulaciones_misma_area
0,0z5Dmrd,Administración,2
1,0z5Dmrd,Almacén / Depósito / Expedición,1
2,0z5Dmrd,Atención al Cliente,1
3,0z5Dmrd,Auditoría,1
4,0z5Dmrd,Contabilidad,1


In [26]:
postulaciones_por_nivel=postulaciones.copy()
postulaciones_por_nivel=postulaciones_por_nivel.groupby(by=['idpostulante','nivel_laboral'],as_index=False).agg({'sepostulo':'count'})
postulaciones_por_nivel=postulaciones_por_nivel.rename(columns={'sepostulo':'postulaciones_mismo_nivel'})
postulaciones_por_nivel.head()

,idpostulante,nivel_laboral,postulaciones_mismo_nivel
0,0z5Dmrd,-,1
1,0z5Dmrd,Gerencia / Alta Gerencia / Dirección,2
2,0z5Dmrd,Junior,3
3,0z5Dmrd,Otro,1
4,0z5Dmrd,Senior / Semi-Senior,14


In [27]:
postulaciones_por_tipo=postulaciones.copy()
postulaciones_por_tipo=postulaciones_por_tipo.groupby(by=['idpostulante','tipo_de_trabajo'],as_index=False).agg({'sepostulo':'count'})
postulaciones_por_tipo=postulaciones_por_tipo.rename(columns={'sepostulo':'postulaciones_mismo_tipo'})
postulaciones_por_tipo.head()

,idpostulante,tipo_de_trabajo,postulaciones_mismo_tipo
0,0z5Dmrd,Full-time,20
1,0z5Dmrd,Part-time,1
2,0z5JW1r,Full-time,25
3,0z5VvGv,Full-time,56
4,0z5VvGv,Part-time,11


In [28]:
postulaciones=pd.merge(postulaciones,postulaciones_por_area,on=['idpostulante','nombre_area_aux'],how='left')
postulaciones=pd.merge(postulaciones,postulaciones_por_tipo,on=['idpostulante','tipo_de_trabajo'],how='left')
postulaciones=pd.merge(postulaciones,postulaciones_por_nivel,on=['idpostulante','nivel_laboral'],how='left')
postulaciones['postulaciones_misma_area']=postulaciones['postulaciones_misma_area'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
postulaciones['postulaciones_mismo_nivel']=postulaciones['postulaciones_mismo_nivel'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
postulaciones['postulaciones_mismo_tipo']=postulaciones['postulaciones_mismo_tipo'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
postulaciones=pd.merge(postulaciones,vistas_por_area,on=['idpostulante','nombre_area_aux'],how='left')
postulaciones=pd.merge(postulaciones,vistas_por_tipo,on=['idpostulante','tipo_de_trabajo'],how='left')
postulaciones=pd.merge(postulaciones,vistas_por_nivel,on=['idpostulante','nivel_laboral'],how='left')
postulaciones['vistas_por_area']=postulaciones['vistas_por_area'].fillna(0)
postulaciones['vistas_por_tipo']=postulaciones['vistas_por_tipo'].fillna(0)
postulaciones['vistas_por_nivel']=postulaciones['vistas_por_tipo'].fillna(0)
postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,edad,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,Doctorado_Abandonado,Doctorado_En Curso,Doctorado_Graduado,...,vistas_postulante,vistas_al_aviso,cantidad,lo_vio,postulaciones_misma_area,postulaciones_mismo_tipo,postulaciones_mismo_nivel,vistas_por_area,vistas_por_tipo,vistas_por_nivel
10767267,1112257327,A3Xb0Zl,0,19.0,True,False,False,False,False,False,...,5.0,0.0,0.000000,False,0,20,13,0.0,5.0,5.0
12338010,1112421794,dYJaKXq,1,27.0,True,False,False,False,False,False,...,11.0,0.0,0.000000,False,28,96,60,2.0,9.0,9.0
1975767,1112280868,e8pjMW,0,33.0,False,True,False,False,False,False,...,25.0,0.0,0.000000,False,1,56,42,0.0,22.0,22.0
6284546,1112301266,Z9WzoV,1,24.0,False,True,False,False,False,False,...,144.0,0.0,0.000000,False,68,20,74,36.0,6.0,6.0
5770301,1111301251,vVjQ9ko,0,25.0,True,False,False,False,False,False,...,5.0,0.0,0.000000,False,0,24,17,0.0,5.0,5.0
8334905,1112277176,LNKk5qB,1,21.0,True,False,False,False,False,False,...,0.0,0.0,0.000000,False,26,61,57,0.0,0.0,0.0
2033638,1112460737,bOje0dq,0,21.0,False,True,False,False,False,False,...,22.0,0.0,0.000000,False,2,23,16,0.0,22.0,22.0
679036,1112368447,63ZBoR,0,32.0,True,False,False,False,False,False,...,34.0,0.0,0.000000,False,0,18,14,0.0,34.0,34.0
11624227,1112287901,VNr8LaD,1,27.0,False,True,False,False,False,False,...,122.0,0.0,0.000000,False,28,244,160,11.0,110.0,110.0
13819891,1112186136,BmZqAEQ,0,28.0,False,True,False,False,False,False,...,0.0,0.0,0.000000,False,0,39,25,0.0,0.0,0.0


In [29]:
train, test=train_test_split(postulaciones.sample(2*(len(postulaciones)//3)),test_size=0.05)
print("Train: ",len(train),"Test: ",len(test))

Train:  10187885 Test:  536205


In [30]:
features=['edad','sexo_FEM', 'sexo_MASC','Doctorado_Abandonado',
'Doctorado_En Curso', 'Doctorado_Graduado', 'Master_Abandonado',
'Master_En Curso', 'Master_Graduado', 'Posgrado_Abandonado',
'Posgrado_En Curso', 'Posgrado_Graduado', 'Secundario_Abandonado',
'Secundario_En Curso', 'Secundario_Graduado', 'Terciario/Técnico_Abandonado',
'Terciario/Técnico_En Curso', 'Terciario/Técnico_Graduado', 'Universitario_Abandonado',
'Universitario_En Curso', 'Universitario_Graduado', 'pide_hombre',
'pide_mujer', 'ingles', 'experiencia', 'paquete_office', 'liderazgo', 'secundario',
'viajar', 'empresa_importante', 'capacitacion', 'remuneracion_pretendida', 'graduados',
'estudiantes', 'conocimientos_especificos', 'vistas_postulante',
'vistas_al_aviso', 'cantidad', 'postulaciones_misma_area', 'postulaciones_mismo_tipo', 
'postulaciones_mismo_nivel', 'lo_vio','vistas_por_area','vistas_por_nivel','vistas_por_tipo']

In [31]:
x_train=train[features]
y_train=train['sepostulo']

x_test=test[features]
y_test=test['sepostulo']

In [32]:
postulaciones=0
c=0
gc.collect()

204

In [33]:
c=MLPClassifier(activation='tanh', alpha=0.0001, batch_size='auto',epsilon=1e-08, hidden_layer_sizes=(len(features), 5), learning_rate='constant',learning_rate_init=0.001, max_iter=200,nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,solver='sgd', tol=0.0001, validation_fraction=0.1, verbose=True,warm_start=False)
#c = joblib.load("../../Data/fiuba_entrenamiento/gian/modelos/NN.pkl")
dt=c.fit(x_train,y_train)

Iteration 1, loss = 0.32525101
Iteration 2, loss = 0.30224391
Iteration 3, loss = 0.29725834
Iteration 4, loss = 0.29475838
Iteration 5, loss = 0.29342343
Iteration 6, loss = 0.29169849
Iteration 7, loss = 0.29037916
Iteration 8, loss = 0.29080547
Iteration 9, loss = 0.29026083
Iteration 10, loss = 0.28995560
Iteration 11, loss = 0.28934813
Iteration 12, loss = 0.28911096
Iteration 13, loss = 0.28825277
Iteration 14, loss = 0.28757831
Iteration 15, loss = 0.28820293
Iteration 16, loss = 0.28715688
Iteration 17, loss = 0.28639680
Iteration 18, loss = 0.28608146
Iteration 19, loss = 0.28555623
Iteration 20, loss = 0.28556572
Iteration 21, loss = 0.28571078
Iteration 22, loss = 0.28563400
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.


In [34]:
from sklearn.metrics import roc_auc_score
x_final=x_test[features]
y_final=c.predict_proba(x_final)
y_pred=[]
for elem in y_final:
    y_pred.append(elem[1])
score=roc_auc_score(y_test, y_pred)
print(score)

0.9462948285833674


In [35]:
joblib.dump(c, "../../Data/fiuba_entrenamiento/gian/modelos/NN v6.pkl")

['../../Data/fiuba_entrenamiento/gian/modelos/NN v6.pkl']

In [38]:
prediccion = pd.read_csv("../../Data/fiuba_entrenamiento/test_final_100k.csv")
print(len(prediccion))
prediccion=pd.merge(prediccion,postulantes,on='idpostulante',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,avisos,on='idaviso',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,avisos_no_one_hot,on='idaviso',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,vistas,on=['idpostulante'],how='left')
prediccion=pd.merge(prediccion,vistas2,on=['idaviso','idpostulante'],how='left')
prediccion=pd.merge(prediccion,vistas3,on=['idaviso','idpostulante'],how='left')
prediccion.cantidad=prediccion.cantidad.fillna(0)
prediccion['vistas_postulante']=prediccion['vistas_postulante'].fillna(0)
prediccion['vistas_al_aviso']=prediccion['vistas_al_aviso'].fillna(0)
prediccion['lo_vio']=(prediccion['vistas_al_aviso']>0)
prediccion=pd.merge(prediccion,postulaciones_por_area,on=['idpostulante','nombre_area_aux'],how='left')
prediccion=pd.merge(prediccion,postulaciones_por_tipo,on=['idpostulante','tipo_de_trabajo'],how='left')
prediccion=pd.merge(prediccion,postulaciones_por_nivel,on=['idpostulante','nivel_laboral'],how='left')
prediccion=pd.merge(prediccion,vistas_por_area,on=['idpostulante','nombre_area_aux'],how='left')
prediccion=pd.merge(prediccion,vistas_por_tipo,on=['idpostulante','tipo_de_trabajo'],how='left')
prediccion=pd.merge(prediccion,vistas_por_nivel,on=['idpostulante','nivel_laboral'],how='left')
prediccion['vistas_por_area']=prediccion['vistas_por_area'].fillna(0)
prediccion['vistas_por_tipo']=prediccion['vistas_por_tipo'].fillna(0)
prediccion['vistas_por_nivel']=prediccion['vistas_por_tipo'].fillna(0)
prediccion['postulaciones_misma_area']=prediccion['postulaciones_misma_area'].fillna(0)
prediccion['postulaciones_mismo_nivel']=prediccion['postulaciones_mismo_nivel'].fillna(0)
prediccion['postulaciones_mismo_tipo']=prediccion['postulaciones_mismo_tipo'].fillna(0)
prediccion.sample(10)

100000
100000
100000
100000


,id,idaviso,idpostulante,edad,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,Doctorado_Abandonado,Doctorado_En Curso,Doctorado_Graduado,...,vistas_postulante,vistas_al_aviso,cantidad,lo_vio,postulaciones_misma_area,postulaciones_mismo_tipo,postulaciones_mismo_nivel,vistas_por_area,vistas_por_tipo,vistas_por_nivel
5786,12684,1112019794,E0Ra0,43.0,True,False,False,False,False,False,...,5.0,0.0,0.0,False,0.0,0.0,14.0,0.0,1.0,1.0
91709,90584,1112468167,eRWd12,37.0,True,False,False,False,False,False,...,12.0,0.0,0.0,False,1.0,17.0,1.0,0.0,0.0,0.0
77485,23993,1112312165,5dDV3z,32.0,True,False,False,False,False,False,...,157.0,0.0,0.0,False,0.0,33.0,23.0,2.0,113.0,113.0
96366,95946,1112491605,1vRqaO,34.0,False,True,False,False,False,False,...,32.0,0.0,0.0,False,0.0,44.0,12.0,0.0,27.0,27.0
47624,82573,1112466207,Yjr345X,29.0,False,True,False,False,False,False,...,0.0,0.0,0.0,False,29.0,104.0,84.0,0.0,0.0,0.0
92930,92051,1112473629,Y0mLdP,42.0,False,True,False,False,False,False,...,8.0,0.0,0.0,False,3.0,19.0,11.0,0.0,0.0,0.0
53389,23274,1112303922,aQ5G85,36.0,True,False,False,False,False,False,...,7.0,0.0,0.0,False,0.0,17.0,15.0,0.0,7.0,7.0
68477,48982,1112437263,elVmB,40.0,False,True,False,False,False,False,...,10.0,0.0,0.0,False,0.0,17.0,15.0,0.0,0.0,0.0
14821,64144,1112457811,4rWlbxE,23.0,True,False,False,False,False,False,...,73.0,0.0,0.0,False,42.0,115.0,59.0,5.0,16.0,16.0
27311,43437,1112425062,EkOQ5z,30.0,False,True,False,False,False,False,...,4.0,0.0,0.0,False,0.0,16.0,0.0,0.0,0.0,0.0


In [39]:
x_final=prediccion[features]
y_final=c.predict_proba(x_final)
y_final

array([[0.84705647, 0.15294353],
       [0.46073908, 0.53926092],
       [0.87490809, 0.12509191],
       ...,
       [0.97168119, 0.02831881],
       [0.96577833, 0.03422167],
       [0.98568531, 0.01431469]])

In [40]:
y_final2=[]
print(c.classes_)
print(len(y_final))
for elem in y_final:
    y_final2.append(elem[1])
print(len(y_final2))

[0 1]
100000
100000


In [41]:
prediccion['sepostulo']=y_final2

In [42]:
prediccion.head()

,id,idaviso,idpostulante,edad,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,Doctorado_Abandonado,Doctorado_En Curso,Doctorado_Graduado,...,vistas_al_aviso,cantidad,lo_vio,postulaciones_misma_area,postulaciones_mismo_tipo,postulaciones_mismo_nivel,vistas_por_area,vistas_por_tipo,vistas_por_nivel,sepostulo
0,0,739260,6M9ZQR,42.0,True,False,False,False,False,True,...,0.0,0.0,False,1.0,19.0,0.0,4.0,24.0,24.0,0.152944
1,1,739260,6v1xdL,30.0,False,True,False,False,False,False,...,0.0,0.0,False,6.0,61.0,0.0,0.0,6.0,6.0,0.539261
2,2,739260,ezRKm9,36.0,True,False,False,False,False,False,...,0.0,0.0,False,4.0,19.0,1.0,1.0,7.0,7.0,0.125092
3,3,758580,1Q35ej,68.0,False,True,False,False,False,False,...,0.0,0.0,False,2.0,22.0,2.0,0.0,0.0,0.0,0.266131
4,4,758580,EAN4J6,32.0,True,False,False,False,False,False,...,0.0,0.0,False,1.0,18.0,1.0,0.0,3.0,3.0,0.019162


In [43]:
prediccion=prediccion[['id','sepostulo']]
prediccion.head()

,id,sepostulo
0,0,0.152944
1,1,0.539261
2,2,0.125092
3,3,0.266131
4,4,0.019162


In [44]:
prediccion.to_csv("../../Data/fiuba_entrenamiento/gian/predicciones/prediccion53.csv",index=False)

In [45]:
prediccion.sepostulo.mean()

0.4853191229846297